In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shipr\\OneDrive\\Documents\\project\\Brain-tumor-Classification\\research'

In [3]:
os.chdir("../")

### Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    """Configuration for Model Training"""
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class MLflowConfig:
    """Configuration for MLflow Tracking"""
    experiment_name: str
    run_name_prefix: str
    tracking_uri: str
    registered_model_name: str
    tags: dict     

### Configuration 

In [ ]:
from BT_Classification.constants import *
from BT_Classification.utils.common import read_yaml, create_directories
from BT_Classification.entity import (
    DataIngestionConfig,PrepareBaseModelConfig,MLflowConfig,TrainingConfig
)


class ConfigurationManager:
    """
    Configuration Manager to read and manage all configurations
    """
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create artifacts root directory
        create_directories([self.config.artifacts_root])
    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Get Data Ingestion configuration
        
        Returns:
            DataIngestionConfig: Configuration object for data ingestion
        """
        config = self.config.data_ingestion
        
        # Create root directory for data ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
            train_data_dir=Path(config.train_data_dir),
            test_data_dir=Path(config.test_data_dir)
        )
        
        return data_ingestion_config
    
     
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Get Base Model Preparation configuration
        
        Returns:
            PrepareBaseModelConfig: Configuration object for base model
        """
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        
        return prepare_base_model_config
    

    def get_training_config(self) -> TrainingConfig:
        """
        Get Training configuration
        
        Returns:
            TrainingConfig: Configuration object for model training
        """
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.data_ingestion.train_data_dir
        
        create_directories([Path(training.root_dir)])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )
        
        return training_config
    
    
    def get_mlflow_config(self) -> MLflowConfig:
        """
        Get MLflow configuration
        
        Returns:
            MLflowConfig: Configuration object for MLflow tracking
        """
        mlflow_config_data = self.config.mlflow
        
        mlflow_config = MLflowConfig(
            experiment_name=mlflow_config_data.experiment_name,
            run_name_prefix=mlflow_config_data.run_name_prefix,
            tracking_uri=mlflow_config_data.tracking_uri if mlflow_config_data.tracking_uri else "",
            registered_model_name=mlflow_config_data.registered_model_name,
            tags=dict(mlflow_config_data.tags) if hasattr(mlflow_config_data, 'tags') else {}
        )
        
        return mlflow_config
        

### 02_model_trainer.py

In [ ]:
import os
import time
from pathlib import Path
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import dagshub
from sklearn.utils import class_weight
import numpy as np
from BT_Classification import logger
from BT_Classification.entity import TrainingConfig, MLflowConfig


class Training:
    """
    FINAL OPTIMIZED Training - Balanced approach based on best results
    """
    
    def __init__(self, config: TrainingConfig, mlflow_config: MLflowConfig):
        self.config = config
        self.mlflow_config = mlflow_config
    
    def get_base_model(self):
        """Load the prepared base model"""
        try:
            logger.info(f"Loading improved model from: {self.config.updated_base_model_path}")
            
            self.model = tf.keras.models.load_model(
                str(self.config.updated_base_model_path)
            )
            
            logger.info("Improved model loaded successfully")
            logger.info(f"Model input shape: {self.model.input_shape}")
            logger.info(f"Model output shape: {self.model.output_shape}")
            
        except Exception as e:
            logger.exception(e)
            raise e
    
    def train_valid_generator(self):
        """Create OPTIMIZED data generators"""
        try:
            logger.info("Setting up OPTIMIZED data generators...")
            
            if self.config.params_is_augmentation:
                logger.info("Optimized augmentation enabled - BALANCED approach")
                train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    shear_range=0.3,
                    zoom_range=0.35,
                    horizontal_flip=True,
                    vertical_flip=True,
                    brightness_range=[0.7, 1.3],
                    fill_mode='nearest',
                    validation_split=0.2
                )
            else:
                train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255,
                    validation_split=0.2
                )
            
            # Training generator
            self.train_generator = train_datagen.flow_from_directory(
                directory=str(self.config.training_data),
                target_size=self.config.params_image_size[:-1],
                batch_size=self.config.params_batch_size,
                class_mode='categorical',
                subset='training',
                shuffle=True,
                seed=42
            )
            
            # Validation generator (no augmentation)
            self.validation_generator = train_datagen.flow_from_directory(
                directory=str(self.config.training_data),
                target_size=self.config.params_image_size[:-1],
                batch_size=self.config.params_batch_size,
                class_mode='categorical',
                subset='validation',
                shuffle=False,
                seed=42
            )
            
            logger.info("="*70)
            logger.info("DATASET INFORMATION")
            logger.info("="*70)
            logger.info(f"Training samples: {self.train_generator.samples}")
            logger.info(f"Validation samples: {self.validation_generator.samples}")
            logger.info(f"Number of classes: {self.train_generator.num_classes}")
            logger.info(f"Class indices: {self.train_generator.class_indices}")
            logger.info(f"Batch size: {self.config.params_batch_size}")
            logger.info("="*70)
            
        except Exception as e:
            logger.exception(e)
            raise e
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Save the trained model"""
        try:
            model.save(str(path))
            logger.info(f"Model saved at: {path}")
        except Exception as e:
            logger.exception(e)
            raise e
    
    def train(self):
        """Train with FINAL OPTIMIZED class weights"""
        try:
            logger.info("="*70)
            logger.info("STARTING FINAL OPTIMIZED TRAINING")
            logger.info("="*70)
            
            steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
            validation_steps = self.validation_generator.samples // self.validation_generator.batch_size
            
            logger.info(f"\nTraining Configuration:")
            logger.info(f"  Epochs: {self.config.params_epochs}")
            logger.info(f"  Batch Size: {self.config.params_batch_size}")
            logger.info(f"  Steps per Epoch: {steps_per_epoch}")
            logger.info(f"  Validation Steps: {validation_steps}")
            
        # ==================== FINAL OPTIMIZED CLASS WEIGHTS ====================
            logger.info("\n" + "="*70)
            logger.info("CALCULATING FINAL OPTIMIZED CLASS WEIGHTS")
            logger.info("="*70)

            # Get base balanced weights
            class_weights_array = class_weight.compute_class_weight(
                class_weight='balanced',
                classes=np.unique(self.train_generator.classes),
                y=self.train_generator.classes
            )

            class_weight_dict = dict(enumerate(class_weights_array))

            # FINAL OPTIMIZED: Based on Run 1 results (which worked best)
            # Analysis: 3x was too weak for Glioma, 6x hurt other classes
            # Optimal: 4.2x Glioma, reduce No Tumor more aggressively
            performance_boost_factors = {
                0: 4.2,  # Glioma: Optimal boost (between 3x and 6x)
                1: 1.0,  # Meningioma: Keep balanced (was perfect at 96%)
                2: 0.45, # No Tumor: Reduce more (was still over-predicting)
                3: 1.6   # Pituitary: Slight increase (86% → target 90%)
            }

            # Apply performance adjustments
            logger.info("Base balanced weights + FINAL OPTIMIZED adjustments:")
            for class_idx, weight in class_weight_dict.items():
                class_name = list(self.train_generator.class_indices.keys())[
                    list(self.train_generator.class_indices.values()).index(class_idx)
                ]
                
                original_weight = weight
                boost_factor = performance_boost_factors.get(class_idx, 1.0)
                adjusted_weight = original_weight * boost_factor
                class_weight_dict[class_idx] = adjusted_weight
                
                logger.info(f"  {class_name} (class {class_idx}): {original_weight:.4f} → {adjusted_weight:.4f} (x{boost_factor})")

            logger.info("="*70)
            logger.info("STRATEGY: Balanced optimization - 4.2x Glioma, control No Tumor")
        # =================================================================
            
            # Initialize MLflow
            logger.info("\n" + "="*70)
            logger.info("MLFLOW: Starting Experiment")
            logger.info("="*70)
            
            dagshub.init(
                repo_owner='rahul22106',
                repo_name='Brain-Tumor-Classification',
                mlflow=True
            )
            
            if self.mlflow_config.tracking_uri:
                mlflow.set_tracking_uri(self.mlflow_config.tracking_uri)
            
            mlflow.set_experiment(self.mlflow_config.experiment_name)
            
            run_name = f"{self.mlflow_config.run_name_prefix}_final_optimized_v4_batch{self.config.params_batch_size}"
            
            with mlflow.start_run(run_name=run_name) as run:
                run_id = run.info.run_id
                logger.info(f"MLflow Run ID: {run_id}")
                logger.info(f"Strategy: FINAL OPTIMIZED - BALANCED PERFORMANCE")
                
                # Log Parameters
                mlflow.log_param("strategy", "final_optimized_balanced_v4")
                mlflow.log_param("model_architecture", "MobileNetV2_Improved")
                mlflow.log_param("batch_size", self.config.params_batch_size)
                mlflow.log_param("epochs", self.config.params_epochs)
                mlflow.log_param("learning_rate", self.config.params_learning_rate)
                mlflow.log_param("dropout", "0.35_0.35_0.3")
                mlflow.log_param("l2_reg", "0.008_0.008_0.005")
                mlflow.log_param("frozen_layers", 100)
                mlflow.log_param("gradient_clipping", 1.0)
                mlflow.log_param("class_weights", str(class_weight_dict))
                mlflow.log_param("augmentation", "optimized_balanced")
                mlflow.log_param("glioma_boost", "4.2x_optimal")
                
                # Set Tags
                for tag_key, tag_value in self.mlflow_config.tags.items():
                    mlflow.set_tag(tag_key, tag_value)
                mlflow.set_tag("training_strategy", "final_optimized_v4")
                mlflow.set_tag("focus", "balanced_all_classes")
                
                # Callbacks
                logger.info("\nSetting up OPTIMIZED callbacks...")

                checkpoint_path = str(self.config.trained_model_path).replace('.keras', '_best.keras')
                checkpoint = tf.keras.callbacks.ModelCheckpoint(
                    filepath=checkpoint_path,
                    monitor='val_accuracy',
                    save_best_only=True,
                    mode='max',
                    verbose=1
                )

                # OPTIMIZED: Early stopping to prevent overfitting
                early_stop = tf.keras.callbacks.EarlyStopping(
                    monitor='val_accuracy',  
                    patience=10,
                    restore_best_weights=True,
                    mode='max',
                    verbose=1,
                    min_delta=0.001
                )
                logger.info("EarlyStopping: monitor=val_accuracy, patience=10, min_delta=0.001")

                reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.3,
                    patience=3,
                    min_lr=1e-7,
                    mode='min',
                    verbose=1
                )
                logger.info("ReduceLROnPlateau: factor=0.3, patience=3")

                csv_logger = tf.keras.callbacks.CSVLogger(
                    filename=str(self.config.root_dir / 'training_log_final_v4.csv'),
                    append=True
                )

                tensorboard = tf.keras.callbacks.TensorBoard(
                    log_dir=str(self.config.root_dir / 'tensorboard_logs_final_v4'),
                    histogram_freq=1
                )
                
                # Balanced performance monitor
                class BalancedMonitor(tf.keras.callbacks.Callback):
                    def on_epoch_end(self, epoch, logs=None):
                        train_acc = logs.get('accuracy', 0)
                        val_acc = logs.get('val_accuracy', 0)
                        gap = train_acc - val_acc
                        
                        if gap > 0.15:
                            logger.warning(f"⚠️ OVERFITTING: Train-Val gap = {gap*100:.1f}%")
                        elif gap < -0.05:
                            logger.warning(f"⚠️ UNDERFITTING: Val > Train by {abs(gap)*100:.1f}%")
                        else:
                            logger.info(f"✓ Good balance")
                        
                        logger.info(f"Epoch {epoch+1}: Train={train_acc*100:.2f}% | Val={val_acc*100:.2f}% | Gap={gap*100:.1f}%")
                        logger.info(f"  Strategy: 4.2x Glioma (optimal), 0.45x No Tumor (controlled)")
                
                balanced_monitor = BalancedMonitor()
                
                # Start Training
                logger.info("\n" + "="*70)
                logger.info("TRAINING STARTED - FINAL OPTIMIZED MODE")
                logger.info("="*70)
                
                start_time = time.time()
                
                history = self.model.fit(
                    self.train_generator,
                    epochs=self.config.params_epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=self.validation_generator,
                    validation_steps=validation_steps,
                    class_weight=class_weight_dict,
                    callbacks=[checkpoint, early_stop, reduce_lr, csv_logger, tensorboard, balanced_monitor],
                    verbose=1
                )
                
                end_time = time.time()
                training_time = end_time - start_time
                
                logger.info("\n" + "="*70)
                logger.info("TRAINING COMPLETED")
                logger.info("="*70)
                logger.info(f"Total training time: {training_time/60:.2f} minutes")
                
                # Log Final Metrics
                final_train_acc = history.history['accuracy'][-1]
                final_val_acc = history.history['val_accuracy'][-1]
                final_train_loss = history.history['loss'][-1]
                final_val_loss = history.history['val_loss'][-1]
                
                best_val_acc = max(history.history['val_accuracy'])
                best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
                
                train_val_gap = final_train_acc - final_val_acc
                
                mlflow.log_metric("final_train_accuracy", final_train_acc)
                mlflow.log_metric("final_val_accuracy", final_val_acc)
                mlflow.log_metric("final_train_loss", final_train_loss)
                mlflow.log_metric("final_val_loss", final_val_loss)
                mlflow.log_metric("best_val_accuracy", best_val_acc)
                mlflow.log_metric("train_val_gap", train_val_gap)
                mlflow.log_metric("training_time_minutes", training_time/60)
                
                # Training Summary
                logger.info("\n" + "="*70)
                logger.info("FINAL TRAINING SUMMARY")
                logger.info("="*70)
                logger.info(f"Final Training Accuracy: {final_train_acc*100:.2f}%")
                logger.info(f"Final Validation Accuracy: {final_val_acc*100:.2f}%")
                logger.info(f"Train-Val Gap: {train_val_gap*100:.1f}%")
                logger.info(f"Best Validation Accuracy: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
                logger.info(f"Class Weights: Glioma=4.2x, Meningioma=1.0x, NoTumor=0.45x, Pituitary=1.6x")
                
                if train_val_gap > 0.1:
                    logger.warning("⚠️ OVERFITTING DETECTED - Consider more regularization")
                elif train_val_gap < -0.05:
                    logger.warning("⚠️ UNDERFITTING - Model can learn more")
                else:
                    logger.info("✓ Good generalization achieved")
                
                # Save Model
                self.save_model(
                    path=self.config.trained_model_path,
                    model=self.model
                )
                
                logger.info(f"\n✓ Best model saved: {checkpoint_path}")
                logger.info(f"✓ Final model saved: {self.config.trained_model_path}")
                
            return history
            
        except Exception as e:
            logger.exception(e)
            raise e
    
    def initiate_model_training(self):
        """Main training execution"""
        try:
            logger.info("="*70)
            logger.info("FINAL OPTIMIZED BALANCED TRAINING PIPELINE")
            logger.info("="*70)
            
            logger.info("\n>>> Step 1: Load Improved Model")
            self.get_base_model()
            
            logger.info("\n>>> Step 2: Setup Optimized Augmentation")
            self.train_valid_generator()
            
            logger.info("\n>>> Step 3: Train with Optimal Weights (4.2x Glioma)")
            history = self.train()
            
            logger.info("\n" + "="*70)
            logger.info("TRAINING COMPLETED - FINAL OPTIMIZED VERSION")
            logger.info("="*70)
            
            return history
            
        except Exception as e:
            logger.exception(e)
            raise e

### Pipeline

In [ ]:

STAGE_NAME = "Model Training Stage"


class ModelTrainingPipeline:
    """
    Pipeline for Model Training Stage
    """
    def __init__(self):
        pass
    
    def main(self):
        """Execute model training pipeline"""
        config = ConfigurationManager()
        training_config = config.get_training_config()
        mlflow_config = config.get_mlflow_config()
        training = Training(config=training_config, mlflow_config=mlflow_config)
        training.initiate_model_training()


if __name__ == '__main__':
    try:
        logger.info(f"\n{'='*70}")
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        logger.info(f"{'='*70}\n")
        
        obj = ModelTrainingPipeline()
        obj.main()
        
        logger.info(f"\n{'='*70}")
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<")
        logger.info(f"{'='*70}\n")
        
    except Exception as e:
        logger.exception(e)
        raise e    
    

### Main.py

In [ ]:

STAGE_NAME = "Model Training Stage"
try:
    logger.info(f"\n{'='*70}")
    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
    logger.info(f"{'='*70}\n")
    
    model_training = ModelTrainingPipeline()
    model_training.main()
    
    logger.info(f"\n{'='*70}")
    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<")
    logger.info(f"{'='*70}\n\n")
    
except Exception as e:
    logger.exception(e)
    raise e



In [ ]:
# Stage 03: Model Training
from BT_Classification.pipeline import ModelTrainingPipeline
STAGE_NAME = "Model Training Stage"
try:
    logger.info(f"\n{'='*70}")
    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
    logger.info(f"{'='*70}\n")
    
    model_training = ModelTrainingPipeline()
    model_training.main()
    
    logger.info(f"\n{'='*70}")
    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<")
    logger.info(f"{'='*70}\n\n")
    
except Exception as e:
    logger.exception(e)
    raise e